In [1]:
# -*- coding: utf-8 -*-
import os; os.environ['KERAS_BACKEND'] = 'theano'
os.environ["THEANO_FLAGS"] = "mode=FAST_RUN,device=cuda,floatX=float32"
import theano

from keras.optimizers import SGD
from keras.layers import Input, Dense, Convolution2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D, Dropout, Flatten, merge, Reshape, Activation, concatenate
from keras.datasets import cifar10
from keras.regularizers import l2
from keras.models import Model

from sklearn.metrics import log_loss

from custom_layers.googlenet_custom_layers import LRN, PoolHelper

import keras
from keras.optimizers import SGD
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import AveragePooling2D, GlobalAveragePooling2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model
import keras.backend as K
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.applications.vgg19 import VGG19
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten

from tqdm import tqdm
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import cv2
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from custom_layers.scale_layer import Scale

import pandas as pd



from subprocess import check_output
K.set_image_dim_ordering('th')

Can not use cuDNN on context None: cannot compile with cuDNN. We got this error:
b"In file included from /tmp/try_flags_rsu773ye.c:3:0:\n/usr/include/stdio.h:365:45: error: expected initializer before '__THROWNL'\n       const char *__restrict __format, ...) __THROWNL;\n                                             ^~~~~~~~~\n/usr/include/stdio.h:380:26: error: expected initializer before '__THROWNL'\n        _G_va_list __arg) __THROWNL;\n                          ^~~~~~~~~\n/usr/include/stdio.h:388:6: error: expected initializer before '__THROWNL'\n      __THROWNL __attribute__ ((__format__ (__printf__, 3, 4)));\n      ^~~~~~~~~\n/usr/include/stdio.h:392:6: error: expected initializer before '__THROWNL'\n      __THROWNL __attribute__ ((__format__ (__printf__, 3, 0)));\n      ^~~~~~~~~\n/usr/include/stdio.h:401:6: error: expected initializer before '__THROWNL'\n      __THROWNL __attribute__ ((__format__ (__printf__, 2, 0))) __wur;\n      ^~~~~~~~~\n/usr/include/stdio.h:404:6: error: exp

In [2]:
def googlenet_model(img_rows, img_cols, channel=1, num_classes=None):
    """
    GoogLeNet a.k.a. Inception v1 for Keras
    Model Schema is based on 
    https://gist.github.com/joelouismarino/a2ede9ab3928f999575423b9887abd14
    ImageNet Pretrained Weights 
    https://drive.google.com/open?id=0B319laiAPjU3RE1maU9MMlh2dnc
    Blog Post: 
    http://joelouismarino.github.io/blog_posts/blog_googlenet_keras.html
    Parameters:
      img_rows, img_cols - resolution of inputs
      channel - 1 for grayscale, 3 for color 
      num_classes - number of class labels for our classification task
    """
    
    input = Input(shape=(channel, img_rows, img_cols))
    conv1_7x7_s2 = Convolution2D(64,7,7,subsample=(2,2),border_mode='same',activation='relu',name='conv1/7x7_s2',W_regularizer=l2(0.0002))(input)
    conv1_zero_pad = ZeroPadding2D(padding=(1, 1))(conv1_7x7_s2)
    pool1_helper = PoolHelper()(conv1_zero_pad)
    pool1_3x3_s2 = MaxPooling2D(pool_size=(3,3),strides=(2,2),border_mode='valid',name='pool1/3x3_s2')(pool1_helper)
    pool1_norm1 = LRN(name='pool1/norm1')(pool1_3x3_s2)
    conv2_3x3_reduce = Convolution2D(64,1,1,border_mode='same',activation='relu',name='conv2/3x3_reduce',W_regularizer=l2(0.0002))(pool1_norm1)
    conv2_3x3 = Convolution2D(192,3,3,border_mode='same',activation='relu',name='conv2/3x3',W_regularizer=l2(0.0002))(conv2_3x3_reduce)
    conv2_norm2 = LRN(name='conv2/norm2')(conv2_3x3)
    conv2_zero_pad = ZeroPadding2D(padding=(1, 1))(conv2_norm2)
    pool2_helper = PoolHelper()(conv2_zero_pad)
    pool2_3x3_s2 = MaxPooling2D(pool_size=(3,3),strides=(2,2),border_mode='valid',name='pool2/3x3_s2')(pool2_helper)
    
    inception_3a_1x1 = Convolution2D(64,1,1,border_mode='same',activation='relu',name='inception_3a/1x1',W_regularizer=l2(0.0002))(pool2_3x3_s2)
    inception_3a_3x3_reduce = Convolution2D(96,1,1,border_mode='same',activation='relu',name='inception_3a/3x3_reduce',W_regularizer=l2(0.0002))(pool2_3x3_s2)
    inception_3a_3x3 = Convolution2D(128,3,3,border_mode='same',activation='relu',name='inception_3a/3x3',W_regularizer=l2(0.0002))(inception_3a_3x3_reduce)
    inception_3a_5x5_reduce = Convolution2D(16,1,1,border_mode='same',activation='relu',name='inception_3a/5x5_reduce',W_regularizer=l2(0.0002))(pool2_3x3_s2)
    inception_3a_5x5 = Convolution2D(32,5,5,border_mode='same',activation='relu',name='inception_3a/5x5',W_regularizer=l2(0.0002))(inception_3a_5x5_reduce)
    inception_3a_pool = MaxPooling2D(pool_size=(3,3),strides=(1,1),border_mode='same',name='inception_3a/pool')(pool2_3x3_s2)
    inception_3a_pool_proj = Convolution2D(32,1,1,border_mode='same',activation='relu',name='inception_3a/pool_proj',W_regularizer=l2(0.0002))(inception_3a_pool)
    inception_3a_output = merge([inception_3a_1x1,inception_3a_3x3,inception_3a_5x5,inception_3a_pool_proj],mode='concat',concat_axis=1,name='inception_3a/output')
    
    inception_3b_1x1 = Convolution2D(128,1,1,border_mode='same',activation='relu',name='inception_3b/1x1',W_regularizer=l2(0.0002))(inception_3a_output)
    inception_3b_3x3_reduce = Convolution2D(128,1,1,border_mode='same',activation='relu',name='inception_3b/3x3_reduce',W_regularizer=l2(0.0002))(inception_3a_output)
    inception_3b_3x3 = Convolution2D(192,3,3,border_mode='same',activation='relu',name='inception_3b/3x3',W_regularizer=l2(0.0002))(inception_3b_3x3_reduce)
    inception_3b_5x5_reduce = Convolution2D(32,1,1,border_mode='same',activation='relu',name='inception_3b/5x5_reduce',W_regularizer=l2(0.0002))(inception_3a_output)
    inception_3b_5x5 = Convolution2D(96,5,5,border_mode='same',activation='relu',name='inception_3b/5x5',W_regularizer=l2(0.0002))(inception_3b_5x5_reduce)
    inception_3b_pool = MaxPooling2D(pool_size=(3,3),strides=(1,1),border_mode='same',name='inception_3b/pool')(inception_3a_output)
    inception_3b_pool_proj = Convolution2D(64,1,1,border_mode='same',activation='relu',name='inception_3b/pool_proj',W_regularizer=l2(0.0002))(inception_3b_pool)
    inception_3b_output = merge([inception_3b_1x1,inception_3b_3x3,inception_3b_5x5,inception_3b_pool_proj],mode='concat',concat_axis=1,name='inception_3b/output')
    
    inception_3b_output_zero_pad = ZeroPadding2D(padding=(1, 1))(inception_3b_output)
    pool3_helper = PoolHelper()(inception_3b_output_zero_pad)
    pool3_3x3_s2 = MaxPooling2D(pool_size=(3,3),strides=(2,2),border_mode='valid',name='pool3/3x3_s2')(pool3_helper)
    
    inception_4a_1x1 = Convolution2D(192,1,1,border_mode='same',activation='relu',name='inception_4a/1x1',W_regularizer=l2(0.0002))(pool3_3x3_s2)
    inception_4a_3x3_reduce = Convolution2D(96,1,1,border_mode='same',activation='relu',name='inception_4a/3x3_reduce',W_regularizer=l2(0.0002))(pool3_3x3_s2)
    inception_4a_3x3 = Convolution2D(208,3,3,border_mode='same',activation='relu',name='inception_4a/3x3',W_regularizer=l2(0.0002))(inception_4a_3x3_reduce)
    inception_4a_5x5_reduce = Convolution2D(16,1,1,border_mode='same',activation='relu',name='inception_4a/5x5_reduce',W_regularizer=l2(0.0002))(pool3_3x3_s2)
    inception_4a_5x5 = Convolution2D(48,5,5,border_mode='same',activation='relu',name='inception_4a/5x5',W_regularizer=l2(0.0002))(inception_4a_5x5_reduce)
    inception_4a_pool = MaxPooling2D(pool_size=(3,3),strides=(1,1),border_mode='same',name='inception_4a/pool')(pool3_3x3_s2)
    inception_4a_pool_proj = Convolution2D(64,1,1,border_mode='same',activation='relu',name='inception_4a/pool_proj',W_regularizer=l2(0.0002))(inception_4a_pool)
    inception_4a_output = merge([inception_4a_1x1,inception_4a_3x3,inception_4a_5x5,inception_4a_pool_proj],mode='concat',concat_axis=1,name='inception_4a/output')
    
    loss1_ave_pool = AveragePooling2D(pool_size=(5,5),strides=(3,3),name='loss1/ave_pool')(inception_4a_output)
    loss1_conv = Convolution2D(128,1,1,border_mode='same',activation='relu',name='loss1/conv',W_regularizer=l2(0.0002))(loss1_ave_pool)
    loss1_flat = Flatten()(loss1_conv)
    loss1_fc = Dense(1024,activation='relu',name='loss1/fc',W_regularizer=l2(0.0002))(loss1_flat)
    loss1_drop_fc = Dropout(0.7)(loss1_fc)
    loss1_classifier = Dense(1000,name='loss1/classifier',W_regularizer=l2(0.0002))(loss1_drop_fc)
    loss1_classifier_act = Activation('softmax')(loss1_classifier)
    
    inception_4b_1x1 = Convolution2D(160,1,1,border_mode='same',activation='relu',name='inception_4b/1x1',W_regularizer=l2(0.0002))(inception_4a_output)
    inception_4b_3x3_reduce = Convolution2D(112,1,1,border_mode='same',activation='relu',name='inception_4b/3x3_reduce',W_regularizer=l2(0.0002))(inception_4a_output)
    inception_4b_3x3 = Convolution2D(224,3,3,border_mode='same',activation='relu',name='inception_4b/3x3',W_regularizer=l2(0.0002))(inception_4b_3x3_reduce)
    inception_4b_5x5_reduce = Convolution2D(24,1,1,border_mode='same',activation='relu',name='inception_4b/5x5_reduce',W_regularizer=l2(0.0002))(inception_4a_output)
    inception_4b_5x5 = Convolution2D(64,5,5,border_mode='same',activation='relu',name='inception_4b/5x5',W_regularizer=l2(0.0002))(inception_4b_5x5_reduce)
    inception_4b_pool = MaxPooling2D(pool_size=(3,3),strides=(1,1),border_mode='same',name='inception_4b/pool')(inception_4a_output)
    inception_4b_pool_proj = Convolution2D(64,1,1,border_mode='same',activation='relu',name='inception_4b/pool_proj',W_regularizer=l2(0.0002))(inception_4b_pool)
    inception_4b_output = merge([inception_4b_1x1,inception_4b_3x3,inception_4b_5x5,inception_4b_pool_proj],mode='concat',concat_axis=1,name='inception_4b_output')
    
    inception_4c_1x1 = Convolution2D(128,1,1,border_mode='same',activation='relu',name='inception_4c/1x1',W_regularizer=l2(0.0002))(inception_4b_output)
    inception_4c_3x3_reduce = Convolution2D(128,1,1,border_mode='same',activation='relu',name='inception_4c/3x3_reduce',W_regularizer=l2(0.0002))(inception_4b_output)
    inception_4c_3x3 = Convolution2D(256,3,3,border_mode='same',activation='relu',name='inception_4c/3x3',W_regularizer=l2(0.0002))(inception_4c_3x3_reduce)
    inception_4c_5x5_reduce = Convolution2D(24,1,1,border_mode='same',activation='relu',name='inception_4c/5x5_reduce',W_regularizer=l2(0.0002))(inception_4b_output)
    inception_4c_5x5 = Convolution2D(64,5,5,border_mode='same',activation='relu',name='inception_4c/5x5',W_regularizer=l2(0.0002))(inception_4c_5x5_reduce)
    inception_4c_pool = MaxPooling2D(pool_size=(3,3),strides=(1,1),border_mode='same',name='inception_4c/pool')(inception_4b_output)
    inception_4c_pool_proj = Convolution2D(64,1,1,border_mode='same',activation='relu',name='inception_4c/pool_proj',W_regularizer=l2(0.0002))(inception_4c_pool)
    inception_4c_output = merge([inception_4c_1x1,inception_4c_3x3,inception_4c_5x5,inception_4c_pool_proj],mode='concat',concat_axis=1,name='inception_4c/output')
    
    inception_4d_1x1 = Convolution2D(112,1,1,border_mode='same',activation='relu',name='inception_4d/1x1',W_regularizer=l2(0.0002))(inception_4c_output)
    inception_4d_3x3_reduce = Convolution2D(144,1,1,border_mode='same',activation='relu',name='inception_4d/3x3_reduce',W_regularizer=l2(0.0002))(inception_4c_output)
    inception_4d_3x3 = Convolution2D(288,3,3,border_mode='same',activation='relu',name='inception_4d/3x3',W_regularizer=l2(0.0002))(inception_4d_3x3_reduce)
    inception_4d_5x5_reduce = Convolution2D(32,1,1,border_mode='same',activation='relu',name='inception_4d/5x5_reduce',W_regularizer=l2(0.0002))(inception_4c_output)
    inception_4d_5x5 = Convolution2D(64,5,5,border_mode='same',activation='relu',name='inception_4d/5x5',W_regularizer=l2(0.0002))(inception_4d_5x5_reduce)
    inception_4d_pool = MaxPooling2D(pool_size=(3,3),strides=(1,1),border_mode='same',name='inception_4d/pool')(inception_4c_output)
    inception_4d_pool_proj = Convolution2D(64,1,1,border_mode='same',activation='relu',name='inception_4d/pool_proj',W_regularizer=l2(0.0002))(inception_4d_pool)
    inception_4d_output = merge([inception_4d_1x1,inception_4d_3x3,inception_4d_5x5,inception_4d_pool_proj],mode='concat',concat_axis=1,name='inception_4d/output')
    
    loss2_ave_pool = AveragePooling2D(pool_size=(5,5),strides=(3,3),name='loss2/ave_pool')(inception_4d_output)
    loss2_conv = Convolution2D(128,1,1,border_mode='same',activation='relu',name='loss2/conv',W_regularizer=l2(0.0002))(loss2_ave_pool)
    loss2_flat = Flatten()(loss2_conv)
    loss2_fc = Dense(1024,activation='relu',name='loss2/fc',W_regularizer=l2(0.0002))(loss2_flat)
    loss2_drop_fc = Dropout(0.7)(loss2_fc)
    loss2_classifier = Dense(1000,name='loss2/classifier',W_regularizer=l2(0.0002))(loss2_drop_fc)
    loss2_classifier_act = Activation('softmax')(loss2_classifier)
    
    inception_4e_1x1 = Convolution2D(256,1,1,border_mode='same',activation='relu',name='inception_4e/1x1',W_regularizer=l2(0.0002))(inception_4d_output)
    inception_4e_3x3_reduce = Convolution2D(160,1,1,border_mode='same',activation='relu',name='inception_4e/3x3_reduce',W_regularizer=l2(0.0002))(inception_4d_output)
    inception_4e_3x3 = Convolution2D(320,3,3,border_mode='same',activation='relu',name='inception_4e/3x3',W_regularizer=l2(0.0002))(inception_4e_3x3_reduce)
    inception_4e_5x5_reduce = Convolution2D(32,1,1,border_mode='same',activation='relu',name='inception_4e/5x5_reduce',W_regularizer=l2(0.0002))(inception_4d_output)
    inception_4e_5x5 = Convolution2D(128,5,5,border_mode='same',activation='relu',name='inception_4e/5x5',W_regularizer=l2(0.0002))(inception_4e_5x5_reduce)
    inception_4e_pool = MaxPooling2D(pool_size=(3,3),strides=(1,1),border_mode='same',name='inception_4e/pool')(inception_4d_output)
    inception_4e_pool_proj = Convolution2D(128,1,1,border_mode='same',activation='relu',name='inception_4e/pool_proj',W_regularizer=l2(0.0002))(inception_4e_pool)
    inception_4e_output = merge([inception_4e_1x1,inception_4e_3x3,inception_4e_5x5,inception_4e_pool_proj],mode='concat',concat_axis=1,name='inception_4e/output')
    
    inception_4e_output_zero_pad = ZeroPadding2D(padding=(1, 1))(inception_4e_output)
    pool4_helper = PoolHelper()(inception_4e_output_zero_pad)
    pool4_3x3_s2 = MaxPooling2D(pool_size=(3,3),strides=(2,2),border_mode='valid',name='pool4/3x3_s2')(pool4_helper)
    
    inception_5a_1x1 = Convolution2D(256,1,1,border_mode='same',activation='relu',name='inception_5a/1x1',W_regularizer=l2(0.0002))(pool4_3x3_s2)
    inception_5a_3x3_reduce = Convolution2D(160,1,1,border_mode='same',activation='relu',name='inception_5a/3x3_reduce',W_regularizer=l2(0.0002))(pool4_3x3_s2)
    inception_5a_3x3 = Convolution2D(320,3,3,border_mode='same',activation='relu',name='inception_5a/3x3',W_regularizer=l2(0.0002))(inception_5a_3x3_reduce)
    inception_5a_5x5_reduce = Convolution2D(32,1,1,border_mode='same',activation='relu',name='inception_5a/5x5_reduce',W_regularizer=l2(0.0002))(pool4_3x3_s2)
    inception_5a_5x5 = Convolution2D(128,5,5,border_mode='same',activation='relu',name='inception_5a/5x5',W_regularizer=l2(0.0002))(inception_5a_5x5_reduce)
    inception_5a_pool = MaxPooling2D(pool_size=(3,3),strides=(1,1),border_mode='same',name='inception_5a/pool')(pool4_3x3_s2)
    inception_5a_pool_proj = Convolution2D(128,1,1,border_mode='same',activation='relu',name='inception_5a/pool_proj',W_regularizer=l2(0.0002))(inception_5a_pool)
    inception_5a_output = merge([inception_5a_1x1,inception_5a_3x3,inception_5a_5x5,inception_5a_pool_proj],mode='concat',concat_axis=1,name='inception_5a/output')
    
    inception_5b_1x1 = Convolution2D(384,1,1,border_mode='same',activation='relu',name='inception_5b/1x1',W_regularizer=l2(0.0002))(inception_5a_output)
    inception_5b_3x3_reduce = Convolution2D(192,1,1,border_mode='same',activation='relu',name='inception_5b/3x3_reduce',W_regularizer=l2(0.0002))(inception_5a_output)
    inception_5b_3x3 = Convolution2D(384,3,3,border_mode='same',activation='relu',name='inception_5b/3x3',W_regularizer=l2(0.0002))(inception_5b_3x3_reduce)
    inception_5b_5x5_reduce = Convolution2D(48,1,1,border_mode='same',activation='relu',name='inception_5b/5x5_reduce',W_regularizer=l2(0.0002))(inception_5a_output)
    inception_5b_5x5 = Convolution2D(128,5,5,border_mode='same',activation='relu',name='inception_5b/5x5',W_regularizer=l2(0.0002))(inception_5b_5x5_reduce)
    inception_5b_pool = MaxPooling2D(pool_size=(3,3),strides=(1,1),border_mode='same',name='inception_5b/pool')(inception_5a_output)
    inception_5b_pool_proj = Convolution2D(128,1,1,border_mode='same',activation='relu',name='inception_5b/pool_proj',W_regularizer=l2(0.0002))(inception_5b_pool)
    inception_5b_output = merge([inception_5b_1x1,inception_5b_3x3,inception_5b_5x5,inception_5b_pool_proj],mode='concat',concat_axis=1,name='inception_5b/output')
    
    pool5_7x7_s1 = AveragePooling2D(pool_size=(7,7),strides=(1,1),name='pool5/7x7_s2')(inception_5b_output)
    loss3_flat = Flatten()(pool5_7x7_s1)
    pool5_drop_7x7_s1 = Dropout(0.4)(loss3_flat)
    loss3_classifier = Dense(1000,name='loss3/classifier',W_regularizer=l2(0.0002))(pool5_drop_7x7_s1)
    loss3_classifier_act = Activation('softmax',name='prob')(loss3_classifier)
    
    # Create model
    model = Model(input=input, output=[loss1_classifier_act,loss2_classifier_act,loss3_classifier_act])
    
    # Load ImageNet pre-trained data 
    model.load_weights('/home/sahand/Projects/Kaggle-dog-breeds-identification/imagenet_models/googlenet_weights.h5')
    
    # Truncate and replace softmax layer for transfer learning
    # Cannot use model.layers.pop() since model is not of Sequential() type
    # The method below works since pre-trained weights are stored in layers but not in the model
    loss3_classifier_statefarm = Dense(num_classes,name='loss3/classifier',W_regularizer=l2(0.0002))(pool5_drop_7x7_s1)
    loss3_classifier_act_statefarm = Activation('softmax',name='prob')(loss3_classifier_statefarm)
    loss2_classifier_statefarm = Dense(num_classes,name='loss2/classifier',W_regularizer=l2(0.0002))(loss2_drop_fc)
    loss2_classifier_act_statefarm = Activation('softmax')(loss2_classifier_statefarm)
    loss1_classifier_statefarm = Dense(num_classes,name='loss1/classifier',W_regularizer=l2(0.0002))(loss1_drop_fc)
    loss1_classifier_act_statefarm = Activation('softmax')(loss1_classifier_statefarm)

    # Create another model with our customized softmax
    model = Model(input=input, output=[loss1_classifier_act_statefarm,loss2_classifier_act_statefarm,loss3_classifier_act_statefarm])

    # Learning rate is changed to 0.001
    sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model 

In [3]:
def snd_data_split_3(X,Y,train_proportion,test_proportion,validation_proportion):
    size_1 = 1 - train_proportion
    size_2 = validation_proportion/(test_proportion+validation_proportion)
    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=size_1)
    x_test, x_valid, y_test, y_valid = train_test_split(x_test, y_test, test_size=size_2)
    return x_train,x_test,x_valid,y_train,y_test,y_valid


In [4]:
    img_rows, img_cols = 224, 224 # Resolution of inputs
    channel = 3
    num_classes = 2 
    batch_size = 64 
    nb_epoch = 30
    
    TrainDataRaw = pd.read_csv('/home/sahand/Desktop/output/log.csv',sep=',',header=None)
    labeled_image_list = TrainDataRaw[0].unique()
    np.random.seed(10)
    
    remove_all = 205000 # Number of data points for dropping
    drop_indices = np.random.choice(TrainDataRaw.index, remove_all, replace=False)
    TrainDataRaw = TrainDataRaw.drop(drop_indices)
    
    TrainDataRaw_0 = TrainDataRaw.loc[TrainDataRaw[4]==0]
    TrainDataRaw_1 = TrainDataRaw.loc[TrainDataRaw[4]==1]
    TrainDataRaw_2 = TrainDataRaw.loc[TrainDataRaw[4]==2]
    
    TrainDataRaw_0 = TrainDataRaw_0.reset_index()
    TrainDataRaw_0 = TrainDataRaw_0.drop("index",axis=1)
    TrainDataRaw_1 = TrainDataRaw_1.reset_index()
    TrainDataRaw_1 = TrainDataRaw_1.drop("index",axis=1)
    
    
    
    remove_n = 17000 # Number of non-cracks for dropping
    drop_indices = np.random.choice(TrainDataRaw_0.index, remove_n, replace=False)
    TrainDataRaw_0 = TrainDataRaw_0.drop(drop_indices)
    
    TrainDataRaw_Full = TrainDataRaw_0.append(TrainDataRaw_1)
    TrainDataRaw_Full = TrainDataRaw_Full.append(TrainDataRaw_2)
    TrainDataRaw_Full = TrainDataRaw_Full.reset_index()
    TrainDataRaw_Full = TrainDataRaw_Full.drop("index",axis=1)
    
    # SHUFFLE DATA
    TrainDataRaw_Full = TrainDataRaw_Full.sample(frac=1).reset_index(drop=True)
    TrainDataRaw_Full = TrainDataRaw_Full.sample(frac=1).reset_index(drop=True)
    
    Y_raw = TrainDataRaw_Full[4] #(5th column= cat)
    X_files = TrainDataRaw_Full[2] #(3th column=directories)
    x_train_raw,x_test_raw,x_valid_raw,y_train,y_test,y_valid = snd_data_split_3(X_files,Y_raw,train_proportion=0.7,test_proportion=0.1,validation_proportion=0.2)

        
    im_size = 224
    x_train = []
    x_valid = []
    x_test = []
#    y_train = []
#    y_valid = []
#    y_test = []
  
    print("generate a tensor of images for train/test/validation sets")
    
    
    for f in tqdm(x_train_raw):
        img = cv2.imread(f)
        x_train.append(cv2.resize(img, (im_size, im_size)))
        
    for f in tqdm(x_test_raw):
        img = cv2.imread(f)
        x_test.append(cv2.resize(img, (im_size, im_size)))
        
    for f in tqdm(x_valid_raw):
        img = cv2.imread(f)
        x_valid.append(cv2.resize(img, (im_size, im_size)))
        

#    for label in tqdm(y_train_r):
#        y_train.append(label)
        
#    for label in tqdm(y_valid_r):
#        y_valid.append(label)
        
#    for label in tqdm(y_test_r):
#        y_test.append(label)    
    
    dummies_created = False
    data_reshaped = False

  2%|▏         | 501/21302 [00:00<00:04, 5009.17it/s]

generate a tensor of images for train/test/validation sets


100%|██████████| 6087/6087 [00:01<00:00, 5196.68it/s]


In [5]:
if dummies_created==False:
    y_train = pd.get_dummies(y_train)
    y_test = pd.get_dummies(y_test)
    y_valid = pd.get_dummies(y_valid)
    dummies_created = True
    print('dummy labels are created')

    y_train = np.array(y_train)
    y_test = np.array(y_test)
    y_valid = np.array(y_valid)

dummy labels are created


In [6]:
if data_reshaped==False:
    y_train = np.array(y_train, np.uint8)
    x_train = np.array(x_train, np.float32) / 255.
    x_train = x_train.reshape(x_train.shape[0],x_train.shape[3],x_train.shape[1],x_train.shape[2])
    print("train data shape")
    print(x_train.shape)
    print(y_train.shape)
    x_test  = np.array(x_test, np.float32) / 255.
    x_test = x_test.reshape(x_test.shape[0],x_test.shape[3],x_test.shape[1],x_test.shape[2])
    print("test data shape")
    print(x_test.shape)
    print(y_test.shape)
    x_valid  = np.array(x_valid, np.float32) / 255.
    x_valid = x_valid.reshape(x_valid.shape[0],x_valid.shape[3],x_valid.shape[1],x_valid.shape[2])
    print("valid data shape")
    print(x_valid.shape)
    print(y_valid.shape) 
    data_reshaped = True
    print('data reshaped and normalized')

train data shape
(21302, 3, 224, 224)
(21302, 2)
test data shape
(3043, 3, 224, 224)
(3043, 2)
valid data shape
(6087, 3, 224, 224)
(6087, 2)
data reshaped and normalized


In [7]:
    # Load our model
    model = googlenet_model(img_rows, img_cols, channel, num_classes)

/home/sahand/anaconda3/envs/py3-theano/lib/python3.6/site-packages/ipykernel_launcher.py:17: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (7, 7), activation="relu", name="conv1/7x7_s2", strides=(2, 2), padding="same", kernel_regularizer=<keras.reg...)`
/home/sahand/anaconda3/envs/py3-theano/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name="pool1/3x3_s2", padding="valid")`
/home/sahand/anaconda3/envs/py3-theano/lib/python3.6/site-packages/ipykernel_launcher.py:22: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (1, 1), activation="relu", name="conv2/3x3_reduce", padding="same", kernel_regularizer=<keras.reg...)`
/home/sahand/anaconda3/envs/py3-theano/lib/python3.6/site-packages/ipykernel_launcher.py:23: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(192, (3, 3), activation="relu", name="conv2/3x3",

/home/sahand/anaconda3/envs/py3-theano/lib/python3.6/site-packages/ipykernel_launcher.py:63: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1024, activation="relu", name="loss1/fc", kernel_regularizer=<keras.reg...)`
/home/sahand/anaconda3/envs/py3-theano/lib/python3.6/site-packages/ipykernel_launcher.py:65: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1000, name="loss1/classifier", kernel_regularizer=<keras.reg...)`
/home/sahand/anaconda3/envs/py3-theano/lib/python3.6/site-packages/ipykernel_launcher.py:68: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(160, (1, 1), activation="relu", name="inception_4b/1x1", padding="same", kernel_regularizer=<keras.reg...)`
/home/sahand/anaconda3/envs/py3-theano/lib/python3.6/site-packages/ipykernel_launcher.py:69: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(112, (1, 1), activation="relu", name="inception_4b/3x3_reduce", padding="same", kernel_regularizer=<keras.reg...)

/home/sahand/anaconda3/envs/py3-theano/lib/python3.6/site-packages/ipykernel_launcher.py:106: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (1, 1), activation="relu", name="inception_4e/5x5_reduce", padding="same", kernel_regularizer=<keras.reg...)`
/home/sahand/anaconda3/envs/py3-theano/lib/python3.6/site-packages/ipykernel_launcher.py:107: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (5, 5), activation="relu", name="inception_4e/5x5", padding="same", kernel_regularizer=<keras.reg...)`
/home/sahand/anaconda3/envs/py3-theano/lib/python3.6/site-packages/ipykernel_launcher.py:108: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(3, 3), strides=(1, 1), name="inception_4e/pool", padding="same")`
/home/sahand/anaconda3/envs/py3-theano/lib/python3.6/site-packages/ipykernel_launcher.py:109: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (1, 1), activation="relu", name="incepti

In [8]:
print(y_valid.shape,
y_test.shape,
y_train.shape)

(6087, 2) (3043, 2) (21302, 2)


In [9]:
    # Start Fine-tuning. 
    # Notice that googlenet takes 3 sets of labels for outputs, one for each auxillary classifier
    model.fit(x_train, [y_train, y_train, y_train],
              batch_size=batch_size,
              epochs=nb_epoch,
              shuffle=True,
              verbose=1,
              validation_data=(x_valid, [y_valid, y_valid, y_valid])
              )

Train on 21302 samples, validate on 6087 samples
Epoch 1/30
21302/21302 [==============================] - 163s - loss: 3.5023 - activation_4_loss: 0.6586 - activation_3_loss: 0.6966 - prob_loss: 0.6152 - activation_4_acc: 0.6944 - activation_3_acc: 0.6755 - prob_acc: 0.7039 - val_loss: 3.3372 - val_activation_4_loss: 0.6023 - val_activation_3_loss: 0.5993 - val_prob_loss: 0.6048 - val_activation_4_acc: 0.7025 - val_activation_3_acc: 0.7025 - val_prob_acc: 0.7025
Epoch 2/30
21302/21302 [==============================] - 162s - loss: 3.3583 - activation_4_loss: 0.6106 - activation_3_loss: 0.6119 - prob_loss: 0.6061 - activation_4_acc: 0.7062 - activation_3_acc: 0.7060 - prob_acc: 0.7070 - val_loss: 3.3615 - val_activation_4_loss: 0.6093 - val_activation_3_loss: 0.6137 - val_prob_loss: 0.6086 - val_activation_4_acc: 0.7025 - val_activation_3_acc: 0.7025 - val_prob_acc: 0.7025
Epoch 3/30
21302/21302 [==============================] - 161s - loss: 3.3564 - activation_4_loss: 0.6110 - activ

21302/21302 [==============================] - 157s - loss: 3.2801 - activation_4_loss: 0.6050 - activation_3_loss: 0.6062 - prob_loss: 0.6047 - activation_4_acc: 0.7066 - activation_3_acc: 0.7070 - prob_acc: 0.7070 - val_loss: 3.2880 - val_activation_4_loss: 0.6088 - val_activation_3_loss: 0.6086 - val_prob_loss: 0.6081 - val_activation_4_acc: 0.7025 - val_activation_3_acc: 0.7025 - val_prob_acc: 0.7025
Epoch 21/30
21302/21302 [==============================] - 157s - loss: 3.2760 - activation_4_loss: 0.6049 - activation_3_loss: 0.6055 - prob_loss: 0.6049 - activation_4_acc: 0.7070 - activation_3_acc: 0.7070 - prob_acc: 0.7070 - val_loss: 3.2833 - val_activation_4_loss: 0.6088 - val_activation_3_loss: 0.6076 - val_prob_loss: 0.6081 - val_activation_4_acc: 0.7025 - val_activation_3_acc: 0.7025 - val_prob_acc: 0.7025
Epoch 22/30
21302/21302 [==============================] - 157s - loss: 3.2724 - activation_4_loss: 0.6049 - activation_3_loss: 0.6053 - prob_loss: 0.6051 - activation_4_ac

In [10]:
    # Make predictions
    predictions_test = model.predict(x_test, batch_size=batch_size, verbose=1)

    # Combine 3 set of outputs using averaging
    predictions_test = sum(predictions_test)/len(predictions_test)

    # Cross-entropy loss score
    score = log_loss(y_test, predictions_test)
    

3043/3043 [==============================] - 6s     


In [11]:
y_test_comparision = pd.DataFrame(y_test)
y_pred_comparison = pd.DataFrame(predictions_test)

print(y_pred_comparison)

             0         1
0     0.697651  0.302349
1     0.697242  0.302758
2     0.697438  0.302562
3     0.698073  0.301927
4     0.697481  0.302519
5     0.697642  0.302358
6     0.697984  0.302016
7     0.697550  0.302450
8     0.697549  0.302451
9     0.697851  0.302149
10    0.699236  0.300764
11    0.697688  0.302312
12    0.697768  0.302232
13    0.697374  0.302626
14    0.697319  0.302681
15    0.697510  0.302490
16    0.697358  0.302642
17    0.697499  0.302501
18    0.697590  0.302410
19    0.698570  0.301430
20    0.697388  0.302612
21    0.697204  0.302796
22    0.697877  0.302123
23    0.698050  0.301950
24    0.697810  0.302191
25    0.698193  0.301807
26    0.697331  0.302669
27    0.697266  0.302734
28    0.697864  0.302136
29    0.697562  0.302438
...        ...       ...
3013  0.697676  0.302324
3014  0.697606  0.302394
3015  0.697083  0.302917
3016  0.698095  0.301905
3017  0.697819  0.302182
3018  0.697040  0.302960
3019  0.698395  0.301605
3020  0.697535  0.302465


In [ ]:
df_confusion = pd.crosstab(y_pred_comparison, y_test_comparision, rownames=['Actual'], colnames=['Predicted'], margins=True)
print(df_confusion)

In [ ]:
if __name__ == '__main__':

    # Example to fine-tune on 3000 samples from Cifar10

    img_rows, img_cols = 224, 224 # Resolution of inputs
    channel = 3
    num_classes = 2 
    batch_size = 16 
    nb_epoch = 10
    
    TrainDataRaw = pd.read_csv('/home/sahand/Desktop/output/log.cvs',sep=',',header=None)
    labeled_image_list = TrainDataRaw[0].unique()
    np.random.seed(10)
    
    remove_all = 0 # Number of non-cracks for dropping
    drop_indices = np.random.choice(TrainDataRaw.index, remove_all, replace=False)
    TrainDataRaw = TrainDataRaw.drop(drop_indices)
    
    TrainDataRaw_0 = TrainDataRaw.loc[TrainDataRaw[4]==0]
    TrainDataRaw_1 = TrainDataRaw.loc[TrainDataRaw[4]==1]
    TrainDataRaw_2 = TrainDataRaw.loc[TrainDataRaw[4]==2]
    
    TrainDataRaw_0 = TrainDataRaw_0.reset_index()
    TrainDataRaw_0 = TrainDataRaw_0.drop("index",axis=1)
    TrainDataRaw_1 = TrainDataRaw_1.reset_index()
    TrainDataRaw_1 = TrainDataRaw_1.drop("index",axis=1)
    
    
    
    remove_n = 1 # Number of non-cracks for dropping
    drop_indices = np.random.choice(TrainDataRaw_0.index, remove_n, replace=False)
    TrainDataRaw_0 = TrainDataRaw_0.drop(drop_indices)
    
    TrainDataRaw_Full = TrainDataRaw_0.append(TrainDataRaw_1)
    TrainDataRaw_Full = TrainDataRaw_Full.append(TrainDataRaw_2)
    TrainDataRaw_Full = TrainDataRaw_Full.reset_index()
    TrainDataRaw_Full = TrainDataRaw_Full.drop("index",axis=1)
    
    # SHUFFLE DATA
    TrainDataRaw_Full = TrainDataRaw_Full.sample(frac=1).reset_index(drop=True)
    TrainDataRaw_Full = TrainDataRaw_Full.sample(frac=1).reset_index(drop=True)
    
    Y_raw = TrainDataRaw_Full[4] #(5th column= cat)
    X_files = TrainDataRaw_Full[2] #(3th column=directories)
    
    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=size_1)
    
    im_size = 224
    x_train = []
    y_train = []
    x_test = []
  
    #generate a tensor of images
    
    i = 0
    for f, breed in tqdm(df_train.values):
        img = cv2.imread(f)
        label = one_hot_labels[i]
        x_train.append(cv2.resize(img, (im_size, im_size)))
        y_train.append(label)
        i += 1


    for f in tqdm(df_test['id'].values):
        img = cv2.imread(directory_path+'test/{}.jpg'.format(f))
        x_test.append(cv2.resize(img, (im_size, im_size)))

    y_train_raw = np.array(y_train, np.uint8)
    x_train_raw = np.array(x_train, np.float32) / 255.
    x_test  = np.array(x_test, np.float32) / 255.

    print(x_train_raw.shape)
    print(y_train_raw.shape)
    print(x_test.shape)

    num_class = y_train_raw.shape[1]
    
    
    

    # Load Cifar10 data. Please implement your own load_data() module for your own dataset
    X_train, X_valid, Y_train, Y_valid = train_test_split(x_train_raw, y_train_raw, test_size=0.1, random_state=1)

    # Load our model
    model = googlenet_model(img_rows, img_cols, channel, num_classes)

    # Start Fine-tuning. 
    # Notice that googlenet takes 3 sets of labels for outputs, one for each auxillary classifier
    model.fit(X_train, [Y_train, Y_train, Y_train],
              batch_size=batch_size,
              nb_epoch=nb_epoch,
              shuffle=True,
              verbose=1,
              validation_data=(X_valid, [Y_valid, Y_valid, Y_valid]),
              )

    # Make predictions
    predictions_valid = model.predict(X_valid, batch_size=batch_size, verbose=1)

    # Combine 3 set of outputs using averaging
    predictions_valid = sum(predictions_valid)/len(predictions_valid)

    # Cross-entropy loss score
    score = log_loss(Y_valid, predictions_valid)